In [1]:
import pandas as pd
import plotly.express as px

A princípio todas as informações da coluna Laptop já estão definidas na outras colunas. Então não será necessário NLP nesse processo

In [2]:
df = pd.read_csv('atividade-1-precificacao-laptops.csv')
with pd.option_context('display.max_colwidth', None):  # more options can be specified also
    display(df.head())

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,"ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core i5-1235U/8GB/512GB SSD/15.6""",New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,"Alurin Go Start Intel Celeron N4020/8GB/256GB SSD/15.6""",New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,"ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core i3-1215U/8GB/256GB SSD/15.6""",New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,"MSI Katana GF66 12UC-082XES Intel Core i7-12700H/16GB/1TB SSD/RTX3050/15.6""",New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,"HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB SSD/15.6""",New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


Visto também que as colunas já estão com os tipos corretamente definidos.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Laptop        2160 non-null   object 
 1   Status        2160 non-null   object 
 2   Brand         2160 non-null   object 
 3   Model         2160 non-null   object 
 4   CPU           2160 non-null   object 
 5   RAM           2160 non-null   int64  
 6   Storage       2160 non-null   int64  
 7   Storage type  2118 non-null   object 
 8   GPU           789 non-null    object 
 9   Screen        2156 non-null   float64
 10  Touch         2160 non-null   object 
 11  Final Price   2160 non-null   float64
dtypes: float64(2), int64(2), object(8)
memory usage: 202.6+ KB


In [32]:
df['Brand'].value_counts()

Brand
Asus                415
HP                  368
Lenovo              366
MSI                 308
Acer                137
Apple               116
Dell                 84
Microsoft            77
Gigabyte             48
Razer                37
Medion               32
LG                   32
Alurin               29
PcCom                24
Samsung              22
Dynabook Toshiba     20
Vant                 11
Primux                8
Deep Gaming           8
Innjoo                6
Thomson               4
Prixton               3
Millenium             2
Denver                1
Jetwing               1
Realme                1
Name: count, dtype: int64

In [33]:
df['Status'].value_counts()

Status
New            1498
Refurbished     662
Name: count, dtype: int64

In [34]:
df['CPU'].value_counts()

CPU
Intel Core i7            710
Intel Core i5            535
AMD Ryzen 7              156
Intel Core i3            130
AMD Ryzen 5              127
Intel Core i9             94
Intel Celeron             94
Intel Evo Core i7         82
AMD Ryzen 9               44
AMD Ryzen 3               44
Intel Evo Core i5         30
Apple M2                  28
AMD 3020e                 13
Apple M2 Pro              13
Apple M1                  11
AMD Athlon                10
Intel Pentium             10
Apple M1 Pro               7
Intel Core M3              5
AMD 3015e                  3
Microsoft SQ1              3
Qualcomm Snapdragon 7      3
AMD Radeon 9               2
Qualcomm Snapdragon 8      2
Intel Evo Core i9          1
Mediatek MT8183            1
AMD 3015Ce                 1
AMD Radeon 5               1
Name: count, dtype: int64

"compact" the different types of GPUs to RTX/GTX/MX/T/AMD/None

In [36]:
df['GPU'].value_counts()

GPU
None                  1371
RTX 3050               129
RTX 3060               122
RTX 3070                97
RTX 4060                62
RTX 3080                51
GTX 1650                50
RTX 4070                40
RTX 4050                33
RTX 2070                25
RTX 2060                20
RTX 4080                18
RTX 4090                17
GTX 1660                14
RTX 2050                11
Radeon Pro 5500M         9
RTX A2000                8
MX 450                   7
RTX A3000                7
RTX 2080                 7
RTX A1000                6
T 1200                   6
T 500                    5
MX 330                   5
MX 550                   5
T 550                    4
RX 6500M                 3
RTX A5500                3
GTX 1050                 3
Radeon Pro RX 560X       3
Radeon Pro 5300M         2
T 600                    2
RX 7600S                 2
610 M                    2
P 500                    1
RX 6700M                 1
MX 130                  

In [4]:
df['Brand'].unique()

array(['Asus', 'Alurin', 'MSI', 'HP', 'Lenovo', 'Medion', 'Acer', 'Apple',
       'Razer', 'Gigabyte', 'Dell', 'LG', 'Samsung', 'PcCom', 'Microsoft',
       'Primux', 'Prixton', 'Dynabook Toshiba', 'Thomson', 'Denver',
       'Deep Gaming', 'Vant', 'Innjoo', 'Jetwing', 'Millenium', 'Realme',
       'Toshiba'], dtype=object)

In [5]:
df[df['Brand']=='Toshiba']

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
1663,Toshiba Tecra A40-J-15b Intel Core i3-1125G4/8...,New,Toshiba,Tecra,Intel Core i3,8,256,SSD,NaN,14.0,No,799.0


In [6]:
df[(df['Brand']=='Dynabook Toshiba') & (df['Model']=='Tecra')]

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
662,Dynabook Toshiba Tecra A50-J-1F8 Intel Core i5...,New,Dynabook Toshiba,Tecra,Intel Core i5,8,256,SSD,NaN,15.6,No,899.01
682,Dynabook Toshiba Tecra A50-K-14S Intel Core i7...,New,Dynabook Toshiba,Tecra,Intel Core i7,16,512,SSD,NaN,15.6,No,1637.00
1120,Dynabook Toshiba Tecra A50-J-1BI Intel Core i5...,Refurbished,Dynabook Toshiba,Tecra,Intel Core i5,8,512,SSD,NaN,15.6,No,959.00
1354,Dynabook Toshiba Tecra A50-J-13Y Intel Core i5...,New,Dynabook Toshiba,Tecra,Intel Core i5,8,512,SSD,NaN,15.6,No,1093.01
1850,Dynabook Toshiba Tecra A40-J-17N Intel Core i5...,Refurbished,Dynabook Toshiba,Tecra,Intel Core i5,8,256,SSD,NaN,14.0,No,889.00


Considerando que o modelo Tecra está em Toshiba e Dynabook Toshiba, renomearei para ter uma marca apenas

In [24]:
df['GPU'].unique()

array(['None', 'RTX 3050', 'RTX 4060', 'RTX 4050', 'RTX 3060', 'RTX 4070',
       'RTX 2050', 'GTX 1650', 'RTX 3070', '610 M', 'RTX 4080',
       'RX 6500M', 'MX 550', 'RTX 3080', 'RTX 4090', 'RX 7600S', 'A 370M',
       'GTX 1660', 'RTX A1000', 'RTX 3000', 'T 1200', 'Radeon Pro 5300M',
       'A 730M', 'Radeon Pro RX 560X', 'RTX A5500', 'Radeon Pro 5500M',
       'T 500', 'T 550', 'RTX A3000', 'T 2000', 'T 600', 'T 1000',
       'Radeon RX 6600M', 'MX 330', 'RTX A2000', 'MX 450', 'RTX 2070',
       'RX 6800S', 'RTX 2080', 'RTX 2060', 'GTX 1050', 'MX 130', 'P 500',
       'RX 6700M', 'GTX 1070'], dtype=object)

In [7]:
df['Brand'] = df['Brand'].replace('Toshiba', 'Dynabook Toshiba')

In [17]:
df['GPU'] = df['GPU'].fillna('None')

In [18]:
px.scatter(df, x='Brand', y='Final Price', color='Model')

In [19]:
px.scatter(df, x='Brand', y='Final Price', color='Status')

In [20]:
px.scatter(df, x='CPU', y='Final Price', color='GPU')

In [22]:
px.scatter(df, x='Touch', y='Final Price', color='Screen')

In [29]:
temp = df.copy()
temp['GPU'] = temp['GPU'].apply(lambda x: 'Yes' if x != 'None' else x)
px.scatter(temp, x='CPU', y='Final Price', color='GPU')